# Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

np.random.seed(1)
OJ = pd.read_csv('https://uppsala.instructure.com/courses/65587/files/3683648/download?download_frd=1')

trainIndex = np.random.choice(OJ.shape[0], size=850, replace=False)
train = OJ.iloc[OJ.index.isin(trainIndex)]
test = OJ.iloc[~OJ.index.isin(trainIndex)]
# 'Lead','Number words female','Total words','Number of words lead','Difference in words lead and co-lead','Number of male actors'
# 'Year','Number of female actors','Number words male','Gross','Mean Age Male','Mean Age Female','Age Lead','Age Co-Lead'
test ['Lead']. replace ('Female', 1 , inplace = True )
test ['Lead']. replace ('Male', 0 , inplace = True )

X_test = test.drop(columns=['Lead'])
y_test = test['Lead']

train ['Lead']. replace ('Female', 1 , inplace = True )
train ['Lead']. replace ('Male', 0 , inplace = True )

X_train = train.drop(columns=['Lead'])
y_train = train['Lead']


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


# Randomforest Classifier

In [ ]:
def performace(y_test,y_pred):
    m1 = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    pre = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print("confusionmatrix:")
    print( m1 )
    print(f"accuracy:{acc}")
    print(f"precision:{pre}")
    print(f"recall:{rec}")
    print(f"F1-score:{f1}")


grid_cv = {
    'n_estimators':[500, 700， 1000， 1200], #list(range(50,1000,150)),
    'max_features': ['sqrt','log2','auto'],
    'max_depth': list(range(5, 10)),
    # 'max_samples': list(range(50, 400, 50)),
    'criterion': ['entropy','gini'],
    'bootstrap': ['True','False']
}

grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=42), grid_cv, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)

model = RandomForestClassifier(n_estimators = grid_search_cv.best_estimator_.n_estimators,
                 max_features = grid_search_cv.best_estimator_.max_features,
                 max_depth = grid_search_cv.best_estimator_.max_depth,
                 criterion = grid_search_cv.best_estimator_.criterion,
                 bootstrap = grid_search_cv.best_estimator_.bootstrap,
                #  max_samples = grid_search_cv.best_estimator_.max_samples,
                 random_state =42)

model.fit(X=X_train, y=y_train)
y_predict = model.predict(X_test)
print(grid_search_cv.best_estimator_)
# print('accuracy rate from randomforest tree is %.2f' %np.mean(y_predict==y_test))
# pd.crosstab(y_predict,y_test)
performace(y_test,y_predict)

Fitting 3 folds for each of 120 candidates, totalling 360 fits
RandomForestClassifier(bootstrap='True', criterion='entropy', max_depth=9,
                       max_features='sqrt', n_estimators=500, random_state=42)
confusionmatrix:
[[137   4]
 [ 27  21]]
accuracy:0.8359788359788359
precision:0.84
recall:0.4375
F1-score:0.5753424657534247


# Bagging Classifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

grid_cv = {
    'n_estimators': list(range(400, 1200, 100)),
    'max_samples': list(range(50, 400, 50)),
    'bootstrap': ['True','False']
}

grid_search_cv = RandomizedSearchCV(BaggingClassifier(random_state=42), grid_cv, scoring = 'neg_log_loss', n_iter=100, verbose=1, cv=5)
grid_search_cv.fit(X_train, y_train)


model = BaggingClassifier(n_estimators = grid_search_cv.best_estimator_.n_estimators,
                 max_samples = grid_search_cv.best_estimator_.max_samples,
                 bootstrap = grid_search_cv.best_estimator_.bootstrap,
                 random_state =42)


model.fit(X=X_train, y=y_train)
X_test = test.drop(columns=['Lead'])
y_test = test['Lead']
y_predict = model.predict(X_test)
print(grid_search_cv.best_estimator_)
print('accuracy rate from randomforest tree is %.2f' %np.mean(y_predict==y_test))
performace(y_test,y_predict)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 49 is smaller than n_iter=100. Running 49 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 49 candidates, totalling 245 fits
BaggingClassifier(max_samples=350, n_estimators=400, random_state=42)
accuracy rate from randomforest tree is 0.84
confusionmatrix:
[[138   5]
 [ 25  21]]
accuracy:0.8412698412698413
precision:0.8076923076923077
recall:0.45652173913043476
F1-score:0.5833333333333333
